# 연습 2 : 측정 오류 완화

현재의 양자 컴퓨터는 다양한 종류의 잡음(노이즈)에 영향을 받습니다. 오류 완화의 기본 원리는 특정 오류 소스의 영향을 줄이는 것입니다. 여기에서는 측정 오류, 즉 올바른 양자 상태를 결정하기 위해 큐빗에서 수행된 측정에서 생기는 오류를 완화하는 방법을 살펴 보겠습니다.

<img src="mitigation.png" width="900"/>
<center>측정 오류 완화</center>

위 그림에서 측정 오류 완화를 적용한 결과를 볼 수 있습니다. 왼쪽의 히스토그램은`ibmq_vigo` 장치를 사용하여 얻은 결과를 보여줍니다. 이상적인 결과는 50 %는 $ 00000 $이고 50 %는 $ 10101 $입니다. 여기에 두 가지 특징이 있습니다.

- 먼저 측정 결과가 $ 00000 $로 치우쳐져 있습니다. 이는 측정 과정에서 큐빗의 에너지 감쇄 때문입니다. 완화는 각 qubit에 대해 $ \vert1 \rangle $ 상태를 $ \vert0 \rangle $ 상태로 만듭니다.
- 둘째, 결과에 $ 00000 $ 및 $ 10101 $ 이외의 다른 개수가 포함되어 있습니다. 이것은 다양한 오류로 인해 발생합니다. 이러한 오류의 한 가지 예는 측정 후 판별에서 비롯되며, 측정에서 얻은 신호는 $ \vert0 \rangle $ 또는 $ \vert1 \rangle $로 식별됩니다.

오른쪽 그림은 결과에서 측정 오류 완화를 수행한 결과를 보여줍니다. 장비 측정수가 $ 00000 $의 $ 50% $ 결과 및 $ 10101 $의 $ 50% $ 결과에 대한 이상적인 기대치에 더 가깝고 반면에 다른 수는 크게 줄은 것을 볼 수 있습니다.


## 측정 오류 완화 작동 방식


우리는 각각 $ 2^n $개의 기저 상태를 준비하고 측정하는 일련의 회로를 만드는 것으로 시작합니다. 여기서 $ n $는 큐빗 수입니다. 예를 들어 $ n=2 $ qubits는 $ | 00 \rangle $, $ | 01 \rangle $, $ | 10 \rangle $ 및 $ | 11 \rangle $ 상태를 개별적으로 준비하여 각각의 측정 결과를 봅니다. 결과 통계는 $ M $ 행렬로 수집되는데, $ M_ {ij} $ 요소는 $ |j\rangle $ 상태가 준비되었을 때 $ |i\rangle $ 출력 상태를 얻을 확률을 제공합니다. 임의의 중첩 상태 $ | \psi \rangle = \sum_j \alpha_j | j \rangle $에서도 양자 역학의 선형성은 노이즈에 의한 출력 상태를 $|\psi_{noisy}\rangle = M |\psi\rangle$로 쓸 수 있게 합니다.

측정 오류 완화의 목표는 노이즈를 모델링하는 것이 아니라 오류를 취소하는 기존 수정을 적용하는 것입니다. 노이즈가 있는 결과가 나오면 측정 오류 완화는 그 결과에 해당하는 초기 상태를 복구하려고합니다. 선형 대수를 사용하면 잡음이 있는 $ |\psi_ {noisy} \rangle $ 결과에 $ M $의 역행렬을 곱해준 결과, 즉 $ | \psi \rangle = M ^ {- 1} | \psi_ {noisy} \rangle $이 노이즈가 없을 때의 상태임을 알 수 있습니다. 측정을 통해 수집된 $M$ 행렬은 대체로 비가역행렬이므로 따라서 일반화된 역행렬 계산이 필요합니다. 또한 잡음은 정해져 있는 것이 아니라 변동이 있으므로 일반적으로 이상적인 잡음없는 상태를 만들 수는 없지만, 그럼에도 이 방법을 통해 어느정도 정답에 근접할 수 있습니다.

Qiskit 교재의 [Chapter 5.2] (https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html)에서 측정 오류 완화에 대한 자세한 설명을 찾을 수 있습니다.

**이 연습의 목표는 노이즈가 없는 결과를 유추하기 위해 노이즈가 많은 결과에 적용 할 수있는 보정 매트릭스 $ M $를 만드는 것입니다. **

---
이 연습을 완료하는 데 유용한 팁과 다른 참가자와의 의사 소통 및 질문에 대한 지침은 다음 [저장소] (https://github.com/qiskit-community/may4_challenge_exercises)를 참조하십시오. 또한이 연습의 사본을 찾을 수 있으므로이 노트북을 자유롭게 편집하고 실험 해보십시오.

---

Qiskit에서 매트릭스의 항목을 대체하여 모든 기본 상태를 테스트하는 회로를 생성하는 것은 다음 코드에 의해 수행됩니다.

In [ ]:
#initialization
%matplotlib inline

# Importing standard Qiskit libraries and configuring account
from qiskit import IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.providers.ibmq import least_busy
from qiskit.tools.jupyter import *
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import *
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter


provider = IBMQ.load_account() # load your IBM Quantum Experience account
# If you are a member of the IBM Q Network, fill your hub, group, and project information to
# get access to your premium devices.
# provider = IBMQ.get_provider(hub='', group='', project='')

from may4_challenge.ex2 import get_counts, show_final_answer

num_qubits = 5
meas_calibs, state_labels = complete_meas_cal(range(num_qubits), circlabel='mcal')

다음으로 이러한 회로를 실제 장비에서 실행하십시오! 여러분이 좋아하는 장비를 선택할 수도 있지만 대기열에서 대기 시간을 줄이려면 사용량이 가장 적은 장비를 선택하는 것이 좋습니다. 다음 셀을 실행하면 선택된 가장 적은 백엔드에 대한 모든 정보를 표시하는 위젯이 표시됩니다. "오류 맵"탭을 클릭하면 장비에 대한 최신 잡음 정보가 표시됩니다. 이 문제에서 중요한 것은 그림의 왼쪽(및 오른쪽)에있는 "판독"(측정) 오류입니다. 각 큐빗에서 수 퍼센트의 판독 오류가 발생하는 것이 일반적입니다. 이것이 우리가 이 연습에서 완화하고있는 오류입니다.

In [ ]:
# find the least busy device that has at least 5 qubits
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= num_qubits and 
                                   not x.configuration().simulator and x.status().operational==True))
backend

다음 셀을 실행하여 위의 모든 단계를 구현하십시오. 가능한 한 변동을 평균내려면 아래에 표시된 것처럼 가장 많은 수의 샷, 즉 `shots = 8192`를 선택하는 것이 좋습니다.

'transpile' 호출은 측정 교정 회로를 사용중인 백엔드의 토폴로지에 매핑합니다. `backend.run ()`은 `job` 인스턴스를 리턴하는 회로를 IBM Quantum 디바이스에 전송하고 `%qiskit_job_watcher`는 제출 된 작업이 파이프 라인에서 어디에 있는지 추적합니다.

In [ ]:
# run experiments on a real device
shots = 8192
experiments = transpile(meas_calibs, backend=backend, optimization_level=3)
job = backend.run(assemble(experiments, shots=shots))
print(job.job_id())
%qiskit_job_watcher

꽤 오랫동안 대기열에서 기다려야 할 수도 있다는 것을 알아두세요. 제출 된 작업을 모니터하기 위해 왼쪽 상단에 표시된 'IBMQ Jobs'창을 펼칠 수 있습니다. 그 동안 다른 작업을 실행할 경우를 대비하여 작업 ID를 보관하십시오. 작업이 끝나면 아래 명령어를 통해 결과에 쉽게 액세스 할 수 있습니다.

```python
job = backend.retrieve_job('YOUR_JOB_ID')
```
    
작업 결과가 나오면 다음 코드를 사용하여 교정 매트릭스 및 교정 플롯을 만들 수 있습니다. 그러나 카운트는 매트릭스 대신 딕셔너리(역주:파이썬 자료 클래스)로 제공되므로 노이즈가 많은 카운트에 직접 적용하여 완화된 카운트가 있는 딕셔너리를 수신 할 수있는 측정 필터 개체를 사용하는 것이 더 편리합니다.

In [ ]:
# get measurement filter
cal_results = job.result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
meas_filter = meas_fitter.filter
#print(meas_fitter.cal_matrix)
meas_fitter.plot_calibration()

교정 플롯에서 올바른 결과들은 대각행렬로 나타나고 모든 잘못된 결과는 대각선 바깥에 나타납니다. 후자의 대부분은 측정하는 동안 상태를 $ | 1 \rangle $에서 $ | 0 \rangle $로 큐빗의 정렬이 풀리는(depolarizing) T1 오류로 인해 매트릭스가 비대칭이됩니다.

아래에서는 4 가지 회로에 대한 노이즈 카운트 배열을 제공합니다. 측정 오류 완화는 장치 별 오류 수정이므로 수신하는 배열은 측정 필터를 생성하기 전에 사용한 백엔드에 따라 다릅니다.

**측정 필터를 적용하여 오류 완화된 데이터를 얻으세요. 이 완화된 데이터를 고려하여 오류가 없는 결과에 가장 가까운 결과를 선택하십시오.**

이 방법으로 수정할 수 없는 다른 유형의 오류가 있으므로 노이즈가 전혀 없는 결과는 얻지 못하지만 완화된 결과의 추세에서 정확한 결과를 추측 할 수 있습니다.

## i) 잡음이 있는 첫번째 세트에 대해 생각해보세요:

In [ ]:
# get noisy counts
noisy_counts = get_counts(backend)
plot_histogram(noisy_counts[0])

In [ ]:
# apply measurement error mitigation and plot the mitigated counts
mitigated_counts_0 = meas_filter.apply(noisy_counts[0])
plot_histogram([mitigated_counts_0, noisy_counts[0]])

## 다음 중 어떤 히스토그램이 같은 회로의 *오류없는* 결과와 가장 닮았습니까?
a) <img src="hist_1a.png" width="500"> 
b) <img src="hist_1b.png" width="500"> 
c) <img src="hist_1c.png" width="500"> 
d) <img src="hist_1d.png" width="500">

In [ ]:
# uncomment whatever answer you think is correct
#answer1 = 'a'
#answer1 = 'b'
#answer1 = 'c'
#answer1 = 'd'

## ii) 잡음이 있는 두 번째 세트에 대해 생각해보세요:

In [ ]:
# plot noisy counts
plot_histogram(noisy_counts[1])

In [ ]:
# apply measurement error mitigation
# insert your code here to do measurement error mitigation on noisy_counts[1]
plot_histogram([mitigated_counts_1, noisy_counts[1]])

## 다음 중 어떤 히스토그램이 같은 회로의 *오류없는* 결과와 가장 닮았습니까?
a) <img src="hist_2a.png" width="500"> 
b) <img src="hist_2b.png" width="500"> 
c) <img src="hist_2c.png" width="500"> 
d) <img src="hist_2d.png" width="500">

In [ ]:
# uncomment whatever answer you think is correct
#answer2 = 'a'
#answer2 = 'b'
#answer2 = 'c'
#answer2 = 'd'

## iii) 다음엔, 잡음이 있는 세 번째 세트에 대해 생각해보세요:

In [ ]:
# plot noisy counts
plot_histogram(noisy_counts[2])

In [ ]:
# apply measurement error mitigation
# insert your code here to do measurement error mitigation on noisy_counts[2]
plot_histogram([mitigated_counts_2, noisy_counts[2]])

## 다음 중 어떤 히스토그램이 같은 회로의 *오류없는* 결과와 가장 닮았습니까?
a) <img src="hist_3a.png" width="500"> 
b) <img src="hist_3b.png" width="500"> 
c) <img src="hist_3c.png" width="500"> 
d) <img src="hist_3d.png" width="500">

In [ ]:
# uncomment whatever answer you think is correct
#answer3 = 'a'
#answer3 = 'b'
#answer3 = 'c'
#answer3 = 'd'

## iv) 마지막으로, 잡음이 있는 네 번째 세트에 대해 생각해보세요:

In [ ]:
# plot noisy counts
plot_histogram(noisy_counts[3])

In [ ]:
# apply measurement error mitigation
# insert your code here to do measurement error mitigation on noisy_counts[3]
plot_histogram([mitigated_counts_3, noisy_counts[3]])

## 다음 중 어떤 히스토그램이 같은 회로의 *오류없는* 결과와 가장 닮았습니까?
a) <img src="hist_4a.png" width="500"> 
b) <img src="hist_4b.png" width="500"> 
c) <img src="hist_4c.png" width="500"> 
d) <img src="hist_4d.png" width="500">

In [ ]:
# uncomment whatever answer you think is correct
#answer4 = 'a'
#answer4 = 'b'
#answer4 = 'c'
#answer4 = 'd'

이 연습문제의 정답은 위 네 문제의 답을 나열하면 됩니다. 연습을 완료하고 진행 상황을 추적하려면 IBM Quantum Challenge 페이지에서 다음 행의 출력을 복사하여 붙여 넣으십시오.

In [ ]:
# answer string
show_final_answer(answer1, answer2, answer3, answer4)

수고하셨습니다. 다음 연습문제로 넘어 가세요!